In [55]:
from src.DataLoader import gdfLoader,ReceiverLoader,RSSIDataLoader
from func.utils.getMarkerDictList import getMarkerDictList
from src.DataLoader import Room,Receiver

In [56]:
#Map Data
MapDataPath="data/cartePalaiseau.csv"
map_gdf=gdfLoader(MapDataPath)
#Receiver Data
ReceiverDataPath="data/carteModulePalaiseau.csv"
receiver_gdf=ReceiverLoader(ReceiverDataPath)

In [57]:
map_gdf.explore(max_zoom=30)

In [58]:
from src.DataLoader import room_dict_generator,room_r_tree_generator
room_dict=room_dict_generator(map_gdf)
room_r_tree=room_r_tree_generator(room_dict)
from src.DataLoader import build_receiver_dict
receiver_dict= build_receiver_dict(receiver_gdf,room_dict,room_r_tree)



In [59]:

from src.Symbolic_Localisation import ZoneSelector
Palaiseau_zone_selector=ZoneSelector(room_r_tree=room_r_tree,room_dict=room_dict)
        

In [60]:
from shapely import equals_exact
def test_case1(receiver_dict):
    receiver=receiver_dict['A8032A31204A']
    rssi=-40
    zone=Palaiseau_zone_selector.zone_selection(receiver,rssi)
    assert len(zone)==1,f'Error: point in more than one room'
    zone=zone[0]
    assert equals_exact(zone,receiver.room.cartesian_polygon),f'Error: {zone} should be {receiver.room.cartesian_polygon}'
test_case1(receiver_dict)
def test_full_treshold(receiver_dict):
    for receiver in receiver_dict.values():
        rssi=-40
        zone=Palaiseau_zone_selector.zone_selection(receiver,rssi)
        assert len(zone)==1,f'Error: point in more than one room'
        zone=zone[0]
        assert equals_exact(zone,receiver.room.cartesian_polygon),f'Error: {zone} should be {receiver.room.cartesian_polygon}'
test_full_treshold(receiver_dict)

In [61]:
from src.visualization import MapCreation

map_object=MapCreation(receiver_dict,room_dict)


In [62]:
import ast
import os
from src.DataLoader import gdfLoader,ReceiverLoader
from src import API
from src.preprocessing import Preprocessor
from src.preprocessing import remove_duplicates_Cleaner,moving_average_Filter,moving_max_Filter,moving_max_averaged_Filter,blank_Filter
import pandas as pd
from src.visualization import rssi_viewer

#API call
callApi=API()
start="2023-07-12 11:00:00.000000"
end="2023-07-12 11:30:00.000000"
mac_module=ast.literal_eval(os.environ['MAC_WEAR_DICT'])['3480']
#"DD84B2E50E06"

raw_data=callApi.getRawDataForCartoWear(mac_module,pd.to_datetime(start),pd.to_datetime(end))
#Preprocessing
sampling_time=1
window='10S'
preprocessor=Preprocessor(raw_data,sampling_time=sampling_time)
rssi_df=preprocessor.set_cleaner(remove_duplicates_Cleaner())\
        .set_filter(moving_max_Filter("2s"))\
        .process()
fig=rssi_viewer(rssi_df)


INFO - data Cleaning


In [63]:
rssi_df.dropna(inplace=True)
rssi_df

,timestamp,macModule,rssi
1,2023-07-12 09:00:09,A8032A311F6A,-75.0
2,2023-07-12 09:00:10,A8032A311F6A,-75.0
3,2023-07-12 09:00:11,A8032A311F6A,-75.0
14,2023-07-12 09:00:22,A8032A311F6A,-75.0
15,2023-07-12 09:00:23,A8032A311F6A,-75.0
...,...,...,...
4427,2023-07-12 09:29:57,A8032A311DAA,-76.0
4429,2023-07-12 09:29:58,A8032A311DAA,-78.0
4430,2023-07-12 09:29:59,A8032A311F96,-78.0
4431,2023-07-12 09:29:59,A8032A311DAA,-78.0


In [64]:
fig=rssi_viewer(rssi_df)


In [65]:
import shapely

def add_element_to_list(list_dict,list_name, element):
    if list_name in list_dict:
        list_dict[list_name].append(element)
    else:
        list_dict[list_name] = [element]
def possible_room_union(zone_list:list)->shapely.geometry.Polygon:
    cartesian_polygon_list=[]
    for zone in zone_list:
        cartesian_polygon_list.append(zone)
    return(shapely.union_all(cartesian_polygon_list))
    

In [66]:
#going through all the data and finding the possible rooms for each timestamp and for each receiver

all_zone={}
for row in rssi_df.iterrows():
    #init
    row=row[1]
    timestamp=row['timestamp']
    #all the possible room according to one receiver
    zone_list=Palaiseau_zone_selector.zone_selection(receiver_dict[row['macModule']],row['rssi'])
    #union of all the possible room according to one of the receiver
    possible_rooms=possible_room_union(zone_list)
    
    #adding this union to the dict with keys: timestamp
    add_element_to_list(all_zone,timestamp,possible_rooms)
res={}
for timestamp,instant_zone in all_zone.items():
    zone=shapely.intersection_all(instant_zone)
    if zone.is_empty:
        zone=shapely.union_all(instant_zone)
    res[timestamp]=zone
res


/home/gchenu/Documents/.venv/lib/python3.10/site-packages/shapely/set_operations.py:178: RuntimeWarning:

invalid value encountered in intersection_all



{Timestamp('2023-07-12 09:00:09'): <POLYGON ((244969.199 6226334.021, 244969.628 6226336.593, 244975.663 622633...>,
 Timestamp('2023-07-12 09:00:10'): <POLYGON ((244969.199 6226334.021, 244969.628 6226336.593, 244975.663 622633...>,
 Timestamp('2023-07-12 09:00:11'): <POLYGON ((244969.199 6226334.021, 244969.628 6226336.593, 244975.663 622633...>,
 Timestamp('2023-07-12 09:00:22'): <POLYGON ((244969.199 6226334.021, 244969.628 6226336.593, 244975.663 622633...>,
 Timestamp('2023-07-12 09:00:23'): <POLYGON ((244969.199 6226334.021, 244969.628 6226336.593, 244975.663 622633...>,
 Timestamp('2023-07-12 09:00:24'): <POLYGON ((244969.199 6226334.021, 244969.628 6226336.593, 244975.663 622633...>,
 Timestamp('2023-07-12 09:00:38'): <POLYGON ((244969.199 6226334.021, 244969.628 6226336.593, 244975.663 622633...>,
 Timestamp('2023-07-12 09:00:39'): <POLYGON ((244969.199 6226334.021, 244969.628 6226336.593, 244975.663 622633...>,
 Timestamp('2023-07-12 09:00:40'): <POLYGON ((244969.199 6226334

transfer the coordinates of the polygon obtained at each timestamp to lat long for plotting on the map

In [67]:
from pyproj import Transformer
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326")
def polygon_to_lat_lon(polygon:shapely.geometry.Polygon)->shapely.geometry.Polygon:
    """
    Return the room in which the point is located.
    
    """
    coords=polygon.exterior.coords
    point_list=[transformer.transform(x,y) for x,y in coords] 
    polygon=shapely.geometry.Polygon(point_list)
    return(polygon)

res_in_lat_long={}
for timestamp,polygon in res.items():
    if isinstance(polygon,shapely.geometry.MultiPolygon):
        polygon_list=[]
        for poly in polygon.geoms:
            # on enleve les elements qui ne sont pas des polygones (line string,multi line string,point)
            if isinstance(poly, shapely.geometry.Polygon):
                polygon_list.append(polygon_to_lat_lon(poly))
        if polygon_list:
            res_in_lat_long[timestamp]=shapely.geometry.MultiPolygon(polygon_list)
        
    elif isinstance(polygon,shapely.geometry.Polygon) :
        res_in_lat_long[timestamp]=polygon_to_lat_lon(polygon)
res_in_lat_long

{Timestamp('2023-07-12 09:00:09'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-07-12 09:00:10'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-07-12 09:00:11'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-07-12 09:00:22'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-07-12 09:00:23'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-07-12 09:00:24'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-07-12 09:00:38'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-07-12 09:00:39'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-07-12 09:00:40'): <POLYGON ((48.713 2.201, 48.7

In [68]:
from  src.visualization import timed_room_Feature
from folium.plugins import TimestampedGeoJson

polygonal_features=timed_room_Feature(res_in_lat_long)

In [69]:
TimestampedGeoJson({"type": "FeatureCollection","features": polygonal_features},
                    period='PT1S',
                    duration='PT1S',
                    transition_time=0,  
                    auto_play=True,date_options='YYYY/MM/DD HH:mm:ss:SSS',
                    ).add_to(map_object)

map_object

In [70]:
map_object.save("output/wip.html")


In [71]:
from src.visualization import MapCreation

map_object=MapCreation(receiver_dict,room_dict)

In [72]:

import folium
from src.utils.geometry import discrete_circle
circle=discrete_circle(6,(48,2),2.5)

geojson={
    "type": "Feature",
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                
            ]
        ]
    }
}
print(geojson)
fg = folium.FeatureGroup(name="circle")
fg.add_child(
    folium.vector_layers.Polygon(
        locations=[[x,y] for x,y in circle.exterior.coords],
        color='blue',
        fill_color="lightblue",
        weight=5,
        popup=(folium.Popup("circle")),
    )
)
map_object.add_child(fg)

map_object


{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[]]}}
